In [1]:
import asyncio
import time
from datetime import datetime
from functools import wraps


def time_logger(func):
    @wraps(func)
    def wrapper(self, *args, **kwargs):
        start = time.time()
        result = func(self, *args, **kwargs)
        end = time.time()
        print(f"[{func.__name__}] executed in: {end-start} seconds")
        return result

    return wrapper


In [2]:
class GoodAsync:
    @time_logger
    async def get_complex_list(self, s):
        a = await self.search_complex(s)
        return a

    @time_logger
    async def search_complex(self, s):
        a = await self.apt_scraper(s)
        print(a)
        return a

    @time_logger
    async def apt_scraper(self, s):
        await asyncio.sleep(s)
        return f"[{datetime.now()}] awake {s}"

In [3]:
class BadAsync:
    @time_logger
    async def get_complex_list(self, s):
        a = await self.search_complex(s)
        return a
        
    @time_logger
    async def search_complex(self, s):
        a = self.apt_scraper(s)
        print(a)
        return a

    @time_logger
    def apt_scraper(self, s):
        time.sleep(s)
        return f"[{datetime.now()}] awake {s}"

In [4]:
good = True
print(f"[{datetime.now()} start time]")
if good:
    good = GoodAsync()
    asyncio.create_task(good.get_complex_list(20))
    asyncio.create_task(good.get_complex_list(30))
    asyncio.create_task(good.get_complex_list(10))
else:
    bad = BadAsync()
    asyncio.create_task(bad.get_complex_list(20))
    asyncio.create_task(bad.get_complex_list(30))
    asyncio.create_task(bad.get_complex_list(10))

# 현재 실행 중인 이벤트 루프 가져오기
current_loop = asyncio.get_running_loop()
print(f"Current event loop: {current_loop}")

# 잠시 대기하여 모든 태스크가 스케줄링되도록 함
await asyncio.sleep(0.1)

# 현재 이벤트 루프의 모든 태스크 가져오기
tasks = asyncio.all_tasks(current_loop)
print(f"Current tasks: {tasks}")


[2024-01-18 16:36:23.905650 start time]
[get_complex_list] executed in: 2.1457672119140625e-06 seconds
[get_complex_list] executed in: 0.0 seconds
[get_complex_list] executed in: 1.1920928955078125e-06 seconds
Current event loop: <_UnixSelectorEventLoop running=True closed=False debug=False>
[search_complex] executed in: 2.1457672119140625e-06 seconds
[apt_scraper] executed in: 7.152557373046875e-07 seconds
[search_complex] executed in: 0.0 seconds
[apt_scraper] executed in: 9.5367431640625e-07 seconds
[search_complex] executed in: 7.152557373046875e-07 seconds
[apt_scraper] executed in: 1.1920928955078125e-06 seconds
Current tasks: {<Task pending name='Task-7' coro=<GoodAsync.get_complex_list() running at /var/folders/0f/3kvptjj52h51rwnrbz5lvf6h0000gn/T/ipykernel_55890/236692989.py:4> wait_for=<Future pending cb=[Task.task_wakeup()]>>, <Task pending name='Task-5' coro=<InteractiveShell.run_cell_async() running at /Users/aohus/Library/Python/3.11/lib/python/site-packages/IPython/core/i

[2024-01-18 16:36:33.906794] awake 10
[2024-01-18 16:36:43.907498] awake 20
[2024-01-18 16:36:53.907429] awake 30


In [5]:
# {
# <Task pending name='Task-1' 
# coro=<main() running at async.py:53> 
# cb=[_run_until_complete_cb() 
#     at /Users/aohus/.pyenv/versions/3.8.3/lib/python3.8/asyncio/base_events.py:184]>, 

# <Task pending name='Task-3' 
# coro=<GoodAsync.get_complex_list() running at async.py:23> 
# wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x10b1bfdf0>()]>>, 

# <Task pending name='Task-4' 
# coro=<GoodAsync.get_complex_list() running at async.py:23> 
# wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x10b1bfdc0>()]>>, 

# <Task pending name='Task-2' 
# coro=<GoodAsync.get_complex_list() running at async.py:23> 
# wait_for=<Future pending cb=[<TaskWakeupMethWrapper object at 0x10b1bfe20>()]>>
# }

In [6]:
# {
# <Task pending name='Task-9' 
# coro=<GoodAsync.get_complex_list() running at /var/folders/0f/3kvptjj52h51rwnrbz5lvf6h0000gn/T/ipykernel_53711/236692989.py:4> 
# wait_for=<Future pending cb=[Task.task_wakeup()]>>, 

# <Task pending name='Task-6' 
# coro=<InteractiveShell.run_cell_async() running at /Users/aohus/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3311> 
# cb=[IPythonKernel._cancel_on_sigint.<locals>.cancel_unless_done(<Future pendi...ernel.py:326]>)() at /Users/aohus/Library/Python/3.11/lib/python/site-packages/ipykernel/ipkernel.py:326, Task.task_wakeup()]>, 

# <Task pending name='Task-8' 
# coro=<GoodAsync.get_complex_list() running at /var/folders/0f/3kvptjj52h51rwnrbz5lvf6h0000gn/T/ipykernel_53711/236692989.py:4> 
# wait_for=<Future pending cb=[Task.task_wakeup()]>>, 

# <Task pending name='Task-7' 
# coro=<GoodAsync.get_complex_list() running at /var/folders/0f/3kvptjj52h51rwnrbz5lvf6h0000gn/T/ipykernel_53711/236692989.py:4> 
# wait_for=<Future pending cb=[Task.task_wakeup()]>>, 

# <Task pending name='Task-4' 
# coro=<Kernel.dispatch_queue() running at /Users/aohus/Library/Python/3.11/lib/python/site-packages/ipykernel/kernelbase.py:542> 
# wait_for=<Task pending name='Task-6' 
#     coro=<InteractiveShell.run_cell_async() running at /Users/aohus/Library/Python/3.11/lib/python/site-packages/IPython/core/interactiveshell.py:3311> 
#     cb=[IPythonKernel._cancel_on_sigint.<locals>.cancel_unless_done(<Future pendi...ernel.py:326]>)() at /Users/aohus/Library/Python/3.11/lib/python/site-packages/ipykernel/ipkernel.py:326, Task.task_wakeup()]> 
#     cb=[_wrap_awaitable.<locals>.<lambda>() at /Users/aohus/Library/Python/3.11/lib/python/site-packages/tornado/gen.py:852, IOLoop.add_future.<locals>.<lambda>() at /Users/aohus/Library/Python/3.11/lib/python/site-packages/tornado/ioloop.py:699]>}
